Testing

In [2]:
# Import modules
import calibration
import photometry
import stacking
import align
import dataReduce
from align import dispFITS
import calibration as calib
import aperE
import size

# Additional Imports
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import pandas as pd

In [3]:
df = pd.read_csv('photometry.csv')

In [5]:
df.head(15)

,X,Y,rX,rY,e,s_e
0,1406.0,570.0,39,47,2.528310e+06,6799.061694
1,509.0,198.0,28,25,3.952561e+05,4098.599342
2,1506.0,201.0,10,14,4.606809e+04,1754.098776
3,959.0,887.0,15,12,7.232805e+04,2027.521311
4,780.0,1251.0,15,16,8.819627e+04,2342.039607
5,1896.0,151.0,3,3,1.458598e+03,463.647591
6,1408.0,572.0,39,86,1.458598e+03,463.647591
7,509.0,198.0,28,50,1.458598e+03,463.647591
8,1508.0,190.0,10,37,1.458598e+03,463.647591
9,960.0,888.0,15,13,1.458598e+03,463.647591


In [4]:
x = np.array(df['X'])
y = np.array(df['Y'])
rX = np.array(df['rX'])
rY = np.array(df['rY'])
flux = np.array(df['e'])
flux_err = np.array(df['s_e'])

rX = np.array([45, 27, 9, 14, 14, 7,
               60, 51, 22, 15, 20, 15])
rY = np.array([43, 24, 13, 10, 14, 10,
               90, 47, 38, 12, 16, 13])

# Quantum Efficiencies
qe_Ha = 0.85
s_qe_Ha = 5/3
qe_oiii = 0.67
s_qe_oiii = 5/3

# The first 6 values are in OIII, the last 6 in Ha
# Divide by quantum efficiency at Ha and OIII wavelengths
# to get photon fluxes
flux_ph = []
flux_err_ph = []

for i in range(len(flux)):
    if i < 6:
        flux_ph.append(flux[i]/qe_oiii)
        flux_err_ph.append(flux_err[i]/qe_oiii)
    else:
        flux_ph.append(flux[i]/qe_Ha)
        flux_err_ph.append(flux_err[i]/qe_Ha)

print(flux_ph)
print(flux_err_ph)


[3777400.6668007504, 579903.0877546992, 61538.5556850154, 96794.02545703015, 122496.76785383027, 27172.76869034148, 21418.535320622108, 21418.535320622108, 21418.535320622108, 21418.535320622108, 21418.535320622108, 21418.535320622108]
[10411.865575770196, 5890.134156571267, 2394.524612577699, 2672.75747151406, 3161.6356959754485, 1937.2415681262373, 1527.0021772289165, 1527.0021772289165, 1527.0021772289165, 1527.0021772289165, 1527.0021772289165, 1527.0021772289165]


In [5]:
A = np.pi*rX*rY
err_A = A*np.sqrt((0.3/rX)**2 + (0.3/rY)**2)
print(A)

A_o = np.array(A[:6])
#flux_err_ph_o = np.array(flux_err_ph[:6])
A_ha = np.array(A[6:])
#flux_err_ph_ha = np.array(flux_err_ph[6:])
err_A_ha = np.array(err_A[6:])
err_A_o = np.array(err_A[:6])
phi_A = A_ha/A_o
err_phi_A = phi_A*np.sqrt((err_A_o/A_o)**2 + (err_A_ha/A_ha)**2)
print(phi_A)
print(err_A)
print(err_phi_A)

[ 6078.9817847   2035.75203953   367.56634047   439.8229715
   615.7521601    219.91148575 16964.60032938  7530.39759065
  2626.3714584    565.48667765  1005.30964915   612.61056745]
[2.79069767 3.69907407 7.14529915 1.28571429 1.63265306 2.78571429]
[ 58.66119902  34.04680656  14.9018824   16.21500112  18.66010834
  11.50440763 101.94456059  65.3647592   41.38325453  18.10440726
  24.13920968  18.70765009]
[0.03172454 0.06970093 0.31079446 0.06277899 0.06312542 0.16874346]


In [6]:
avg_phi_A = np.average(phi_A)
#err_phi_A = np.std(phi_A)
print(avg_phi_A)
#print(err_phi_A)

3.2231920877408142


In [7]:
err_phi_A_avg = 1/len(phi_A)*np.sqrt(np.sum(err_phi_A**2))
print(err_phi_A_avg)

0.06210614488329605


In [8]:
flux_ph = [3777400.6668007504,579903.0877546992,61538.5556850154,
           96794.02545703015,122496.76785383027,27172.76869034148,
           13214808.347748976,3761566.502401952,500522.6281330024,
           118320.00290572096,120993.99365846878,112986.3149224409]

flux_err_ph = [113237.91379370152,18285.198198367103,3017.9817024727377,
               3936.00228687029,4833.92279933649,2100.198226496646,
               500702.6487489192,143046.29227412748,19786.695077205055,
               5058.338826643179,5499.057158549219,4963.432759348813]

flux_ph_o = np.array(flux_ph[:6])
flux_err_ph_o = np.array(flux_err_ph[:6])
flux_ph_ha = np.array(flux_ph[6:])
flux_err_ph_ha = np.array(flux_err_ph[6:])

In [9]:
phi_F = flux_ph_ha/flux_ph_o
phi_err_F = phi_F*np.sqrt((flux_err_ph_ha/flux_ph_ha)**2 + (flux_err_ph_o/flux_ph_o)**2)
print(phi_F)
print(phi_err_F)

[3.49838673 6.48654332 8.13348026 1.22238953 0.98773213 4.15807149]
[0.1690224  0.32043732 0.5123392  0.07212319 0.05945157 0.36966239]


In [10]:
avg_phi_F = np.average(phi_F)
#err_phi_F = np.std(phi_F)
err_phi_F_avg = 1/len(phi_F)*np.sqrt(np.sum(phi_err_F**2))
print(err_phi_F_avg)
print(avg_phi_F)


0.12237540440856745
4.0811005775732
